In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [80]:
url_inicial = 'https://critica.filosoficas.unam.mx/index.php/critica/issue/archive/7'
url_root = 'https://critica.filosoficas.unam.mx/index.php/critica/issue/archive/7'
r=requests.get(url_inicial)

In [81]:
soup = BeautifulSoup(r.text, 'html.parser')

In [82]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div', class_='issue-summary media')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://critica.filosoficas.unam.mx/index.php/critica/issue/view/7',
 'https://critica.filosoficas.unam.mx/index.php/critica/issue/view/6',
 'https://critica.filosoficas.unam.mx/index.php/critica/issue/view/5',
 'https://critica.filosoficas.unam.mx/index.php/critica/issue/view/4',
 'https://critica.filosoficas.unam.mx/index.php/critica/issue/view/3',
 'https://critica.filosoficas.unam.mx/index.php/critica/issue/view/2',
 'https://critica.filosoficas.unam.mx/index.php/critica/issue/view/1']

In [83]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [84]:
vol2

['https://critica.filosoficas.unam.mx/index.php/critica/article/view/58',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/59',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/60',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/61',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/62',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/63',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/50',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/51',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/52',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/53',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/42',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/43',
 'https://critica.filosoficas.unam.mx/index.php/critica/article/view/44',
 'https://critica.filosoficas.unam.mx/

In [85]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [86]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://critica.filosoficas.unam.mx/index.php/critica/issue/archive/7


In [87]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

31

In [88]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [89]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Crítica: Revista Hispanoamericana de Filosofía'
    a='IV. Humanidades y Ciencias de la Conducta'
    tem='Filosofía del Lenguaje, Lógica, Ética, Metafísica, Epistemología, Filosofía de la Ciencia, Filosofía de la Mente, Filosofía del Derecho, Estética'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-summary').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [90]:
list_scraper=list_scraper[0:107]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30


In [91]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...",Los ingredientes de la percepción,,None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...
1,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...",Extensionality and Propositional Identity,1.La falacia del extensionalismo. Algunos lógi...,None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...
2,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...","On Promising and its Justice, and Whether it N...","Muchas acciones humanas son, sin duda, acontec...",None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...
3,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...",Confusiones axiológicas,,None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...
4,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...",Differential and Projectible Predicates,"En un artículo reciente (""Some Thoughts on Goo...",None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...
5,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...",Sobre las paradojas de autorreferencia,,None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...
6,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...",Deontic Logic and the Theory of Conditions,La lógica deóntica comenzó siendo una rama de ...,None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...
7,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...","Black vs. Tarski en el ""Problema filosófico de...",,None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...
8,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...",Aesthetics and the Philosophy of Art,Se trata de exponer algunas razones para recha...,None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...
9,Crítica: Revista Hispanoamericana de Filosofía,IV. Humanidades y Ciencias de la Conducta,"Filosofía del Lenguaje, Lógica, Ética, Metafís...","Sobre el concepto de objeto en el ""Tractatus""",,None,https://critica.filosoficas.unam.mx/index.php/...,https://critica.filosoficas.unam.mx/index.php/...


In [92]:
df_catalogo.to_csv('Revista53.07.csv', index=False)